# Traffic Signal State Detection
This is a three stage process to detect the state of a traffic signal for auto-start/stop of cars on signals 

### Imports

In [2]:
import cv2
import numpy as np
from PIL import Image, ImageDraw
from math import atan2, sqrt, pi, cos, sin
from collections import defaultdict
import random as rng

### Color Matching

In [3]:
def RedSignalIntensityBounds():
    return ([4, 0, 200], [150, 80, 246])

In [4]:
def YellowSignalIntensityBounds():
    return ([0, 160, 186], [46, 220, 255])

In [5]:
def GreenSignalIntensityBounds():
    return ([0, 100, 0], [80, 230, 56])

In [6]:
def ColorBounding(image,  color):
    height, width, alp = image.shape
    if color == "Red":
        lower, upper = RedSignalIntensityBounds()
    if color == "Yellow":
        lower, upper = YellowSignalIntensityBounds()
    if color == "Green":
        lower, upper = GreenSignalIntensityBounds()
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
    
    mask = cv2.inRange(image, lower, upper)
    output = cv2.bitwise_and(image, image, mask = mask)
    cropped = []
    x = 0
    for i in output:
        x = x + 1
        y = 0
        for j in i:
            y = y+1
            if j[0]!=0 and j[1]!=0 and j[2]!=0:
                cropped.append([y,x])
                #print("x at "+str(x)+" y at "+str(y))
    #print(len(cropped))
    if len(cropped) != 0:
        min_y = min(cropped, key = lambda x : x[1])
        max_y = max(cropped, key = lambda x : x[1])
        min_x = min(cropped, key = lambda x : x[0])
        max_x = max(cropped, key = lambda x : x[0])
        x_min_coordinate = min_x[0]
        y_min_coordinate = min_y[1]
        x_max_coordinate = max_x[0]
        y_max_coordinate = max_y[1]
        #print (min_x,min_y,max_x,max_y)
        crop_img = image[y_min_coordinate-25:y_max_coordinate+25,x_min_coordinate-25:x_max_coordinate+25]
        #print(crop_img)
        cv2.imwrite("colorboundingoutput.jpg",crop_img)
        return (True, crop_img, x_min_coordinate, y_min_coordinate, x_max_coordinate, y_max_coordinate)
    if len(cropped) == 0:
        return (False, cropped, -1, -1, -1, -1)

### Hough Transform

In [7]:
def GAUSS_BLUR(pixels, w, h):
    kernel = (1/256) * np.array([[1,  4,  6,  4, 1],
                                 [4, 16, 24, 16, 4],
                                 [6, 24, 36, 24, 6], 
                                 [4, 16, 24, 16, 4],
                                 [1,  4,  6,  4, 1]])

    mid = len(kernel) // 2
    share = lambda x, l, u: l if x < l else u if x > u else x
    blurred = np.empty((w, h))
    for x in range(w):
        for y in range(h):
            adder = 0
            for a in range(len(kernel)):
                for b in range(len(kernel)):
                    xn = share(x + a - mid, 0, w - 1)
                    yn = share(y + b - mid, 0, h - 1)
                    adder += pixels[xn, yn] * kernel[a, b]
            blurred[x, y] = int(adder)
    return blurred

In [8]:
def to_GRAY(pixels, w, h):
    gray = np.empty((w, h))
    for x in range(w):
        for y in range(h):
            pixel = pixels[x, y]
            gray[x, y] = (pixel[0] + pixel[1] + pixel[2]) / 3
    return gray


In [9]:
def CANNY(pixels, w, h):
    gray = to_GRAY(pixels, w, h)
    blur = GAUSS_BLUR(gray, w, h)
    gradient, direction = GRADIENT(blur, w, h)
    Filter1(gradient, direction, w, h)
    res = Filter2(gradient, w, h, 20, 25)
    return res

In [10]:
def GRADIENT(pixels, w, h):
    direction = np.zeros((w, h))
    gradient = np.zeros((w, h))
    for x in range(w):
        for y in range(h):
            if 0 < x < w - 1 and 0 < y < h - 1:
                magx = pixels[x + 1, y] - pixels[x - 1, y]
                magy = pixels[x, y + 1] - pixels[x, y - 1]
                gradient[x, y] = sqrt(magx**2 + magy**2)
                direction[x, y] = atan2(magy, magx)
    return gradient, direction

In [11]:
def Filter1(gradient, direction, w, h):
    for x in range(1, w - 1):
        for y in range(1, h - 1):
            if direction[x, y] >= 0:
                angle = direction[x, y]
            else:
                angle = direction[x, y] + pi
            rangle = round(angle / (pi / 4))
            mag = gradient[x, y]
            if ((rangle == 0 or rangle == 4) and (gradient[x - 1, y] > mag or gradient[x + 1, y] > mag)
                    or (rangle == 1 and (gradient[x - 1, y - 1] > mag or gradient[x + 1, y + 1] > mag))
                    or (rangle == 2 and (gradient[x, y - 1] > mag or gradient[x, y + 1] > mag))
                    or (rangle == 3 and (gradient[x + 1, y - 1] > mag or gradient[x - 1, y + 1] > mag))):
                gradient[x, y] = 0

In [12]:
def Filter2(gradient, w, h, low, high):
    holder1 = set()
    for x in range(w):
        for y in range(h):
            if gradient[x, y] > high:
                holder1.add((x, y))
    copy = holder1
    while copy:
        holder2 = set()
        for x, y in copy:
            for a, b in ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)):
                if gradient[x + a, y + b] > low and (x+a, y+b) not in holder1:
                    holder2.add((x+a, y+b))
        holder1.update(holder2)
        copy = holder2

    return list(holder1)

In [13]:
def HOUGH_T(tempImage, rmin, rmax, steps, th):
    image = Image.open(tempImage)
    coord = []
    for r in range(rmin, rmax + 1):
        for t in range(steps):
            coord.append((r, int(r * cos(2 * pi * t / steps)), int(r * sin(2 * pi * t / steps))))

    acc = defaultdict(int)
    pixels = image.load()
    w = image.width
    h = image.height
    cannyOutput = CANNY(pixels, w, h)
    for x, y in cannyOutput:
        for r, dx, dy in coord:
            a = x - dx
            b = y - dy
            acc[(a, b, r)] += 1

    circles = []
    for k, v in sorted(acc.items(), key=lambda i: -i[1]):
        x, y, r = k
        if v / steps >= th and all((x - xc) ** 2 + (y - yc) ** 2 > rc ** 2 for xc, yc, rc in circles):
            #print(v / steps, x, y, r)
            circles.append((x, y, r))
    
    output_image = Image.new("RGB", image.size)
    output_image.paste(image)
    draw_result = ImageDraw.Draw(output_image)
    for x, y, r in circles:
        draw_result.ellipse((x-r, y-r, x+r, y+r), outline=(255,0,0,0))

    # Save output image
    output_image.save("result.jpg")
    
    if len(circles) > 0:
        return True
    else:
        return False

In [22]:
def TrafficLightObjectDetection(img, xminCir, xmaxCir, yminCir, ymaxCir):
    bounded_circle = False
    bounded_Rectangle = False
    edges = cv2.Canny(img,100,200)
    contours,h = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        per = cv2.arcLength(c,True)
        app = cv2.approxPolyDP(c,0.05*per, True)
        contours_poly[i] = app
        if len(app)>3:
            boundRect[i] = cv2.boundingRect(contours_poly[i])
            centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])

    drawing = np.zeros((edges.shape[0], edges.shape[1], 3), dtype=np.uint8)
    # Draw polygonal contour + bonding rects + circles
    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        if len(contours_poly[i]) >3:
            cv2.drawContours(edges, contours_poly, i, color)
            cv2.rectangle(edges, (int(boundRect[i][0]), int(boundRect[i][1])), \
                          (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
            cv2.circle(edges, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)
            #print(int(boundRect[i][0]), int(boundRect[i][1]),int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3]))
            xminRect = int(boundRect[i][0])-10
            xmaxRect = int(boundRect[i][0]+boundRect[i][2])+10
            yminRect = int(boundRect[i][1])
            ymaxRect = int(boundRect[i][1]+boundRect[i][3])
            #print("YE")
            #print(xminCir)
            #print(xmaxCir)
            #print(yminCir)
            #print(ymaxCir)
            #print("YESS")
            #print(xminRect)
            #print(xmaxRect)
            #print(yminRect)
            #print(ymaxRect)
            #print("CIRCRC")
            #print((int(centers[i][0])-int(radius[i])))
            #print((int(centers[i][0])+int(radius[i])))
            #print((int(centers[i][1])-int(radius[i])))
            #print((int(centers[i][1])+int(radius[i])))
            if ((xminCir + xmaxCir)/2) >= xminRect and ((xminCir + xmaxCir)/2) <= xmaxRect and ((yminCir + ymaxCir)/2) <= ymaxRect and ((yminCir + ymaxCir)/2) >= yminRect:
                bounded_Rectangle = True
                #print("MMMMMMMMMMM.")
            if ((xminCir + xmaxCir)/2) >= (int(centers[i][0])-int(radius[i])) and ((xminCir + xmaxCir)/2) <= (int(centers[i][0])+int(radius[i])) and ((yminCir + ymaxCir)/2) >= (int(centers[i][1])-int(radius[i])) and ((yminCir + ymaxCir)/2) <= (int(centers[i][1])+int(radius[i])):
                bounded_circle = True
                #print("LOLOLOLOLO")
    cv2.imwrite("houghoutput.jpg", edges)
    if bounded_circle or bounded_Rectangle:
        return True
    return False

### Driver Program

In [32]:
def TrafficSignalStateDetection(image):
    trafficState = "None"
    isRed, crop_img, xmin, ymin, xmax, ymax = ColorBounding(image, "Red")
    #print(isRed)
    if isRed:
        trafficState = "Red"
    if trafficState == "None" or trafficState == "Red":
        isYellow, crop_img_yellow, xmin_yellow, ymin_yellow, xmax_yellow , ymax_yellow = ColorBounding(image, "Yellow")
        #print(isYellow)
        if isYellow:
            trafficState = "Yellow"
            crop_img = crop_img_yellow
            xmin = xmin_yellow
            ymin = ymin_yellow
            xmax = xmax_yellow
            ymax = ymax_yellow
    if trafficState == "None":
        isGreen, crop_img, xmin, ymin, xmax, ymax = ColorBounding(image, "Green")
        #print(isGreen)
        if isGreen:
            trafficState = "Green"
    if trafficState == "None":
        print("No Traffic Signals Detected!")
    if trafficState != "None":
        houghoutput = HOUGH_T("colorboundingoutput.jpg", 10, 100, 120, 0.3)
        if houghoutput:
            finalOutput = TrafficLightObjectDetection(image, xmin, xmax, ymin, ymax)
            #print(finalOutput)
            if finalOutput:
                print("Current traffic Signal state: " + trafficState)
                return trafficState
            else:
                print("Circle with color " + trafficState + " detected but not a traffic signal")
        else:
            print("No Traffic Signals Detected but " + trafficState + " color detected!")
    return "None"

### Image Signal Detection

In [36]:
img = cv2.imread('red.jpg')
TrafficSignalStateDetection(img)

Current traffic Signal state: Red


'Red'

### Video Signal Detection

In [34]:

vidcap = cv2.VideoCapture('four.mp4')
success,image = vidcap.read()
count = 0
font                   = cv2.FONT_HERSHEY_SIMPLEX
upperLeftCornerOfText = (100,100)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2
while success:
  #TrafficSignalStateDetection(image)
  vidcap.set(cv2.CAP_PROP_POS_MSEC, (count*1000))
  #print('Read a new frame: ', success)
  
  state = TrafficSignalStateDetection(image)
  print(state)
  cv2.putText(image,'State: '+state, upperLeftCornerOfText, font, fontScale, fontColor, lineType)
  cv2.imwrite("frames/frame"+str(count)+".jpg", image)# save frame as JPEG file
  
  success,image = vidcap.read()
  count += 1
#print(count)


Current traffic Signal state: Red
Red
Current traffic Signal state: Red
Red
Current traffic Signal state: Red
Red
Current traffic Signal state: Red
Red
Current traffic Signal state: Yellow
Yellow
Current traffic Signal state: Yellow
Yellow
Current traffic Signal state: Green
Green


### Stitching all frames together for a Video

In [35]:
import os
files = [f for f in os.listdir("frames")]
files.sort(key = lambda x: int(x[5:-4]))
image_array = []
for i in range(len(files)):
    img = cv2.imread("frames/"+files[i])
    size = (img.shape[1], img.shape[0])
    img = cv2.resize(img, size)
    image_array.append(img)
fou = cv2.VideoWriter_fourcc('D','I','V','X')
out = cv2.VideoWriter("video/video.mp4", fou, 1, size)
for i in range(len(image_array)):
    out.write(image_array[i])
out.release()
    